In [36]:
from dataclasses import dataclass

@dataclass(frozen=True)
class GrassCutterState:
    size: int
    grass_position: tuple[tuple[int, int]]
    current_row: int = 0
    current_col: int = 0
    move_times: int = 0

    def __repr__(self):
        state_array = []
        for row in range(self.size):
            row_array = []
            for col in range(self.size):
                has_grass = (row, col) in self.grass_position
                is_robot = (row, col) == (self.current_row, self.current_col)
                if has_grass and is_robot:
                    row_array.append('R')
                elif has_grass:
                    row_array.append('G')
                elif is_robot:
                    row_array.append('r')
                else:
                    row_array.append(' ')
            state_array.append(row_array)
        return '\n'.join(['|'.join(row) for row in state_array]) + f'\nMove times: {self.move_times}'
    
    def __hash__(self):
        return hash((self.size, self.grass_position, self.current_row, self.current_col))
    
    def __eq__(self, other):
        return self.__hash__() == other.__hash__()
    
    def __lt__(self, other):
        return self.move_times < other.move_times

In [16]:
# Create environment
import random

class GrassCutterEnvironment:

    move_str_to_delta = {
        'U': (-1, 0),
        'D': (1, 0),
        'L': (0, -1),
        'R': (0, 1)
    }
    
    @classmethod
    def create_init_state(cls, size: int, random_seed=4242) -> GrassCutterState:
        random.seed(random_seed)
        grass_position = tuple([(random.randint(0, size-1), random.randint(0, size-1)) for _ in range(size)])
        return GrassCutterState(size, grass_position)
    
    @classmethod
    def get_valid_moves(cls, state: GrassCutterState) -> list[str]:
        valid_moves = ['C']
        if state.current_row > 0:
            valid_moves.append('U')
        if state.current_row < state.size - 1:
            valid_moves.append('D')
        if state.current_col > 0:
            valid_moves.append('L')
        if state.current_col < state.size - 1:
            valid_moves.append('R')
        return valid_moves
    
    @classmethod
    def apply_move(cls, state: GrassCutterState, move: str) -> GrassCutterState:
        if move == 'C':
            new_grass_position = tuple([pos for pos in state.grass_position if pos != (state.current_row, state.current_col)])
            return GrassCutterState(state.size, new_grass_position, state.current_row, state.current_col, state.move_times + 1)
        else:
            delta_row, delta_col = cls.move_str_to_delta[move]
            new_row = state.current_row + delta_row
            new_col = state.current_col + delta_col
            return GrassCutterState(state.size, state.grass_position, new_row, new_col, state.move_times + 1)
        
    @classmethod
    def is_terminal(cls, state: GrassCutterState) -> bool:
        return len(state.grass_position) == 0
    

In [26]:
import time
from concurrent.futures import ThreadPoolExecutor

def run_simulation(size: int, total_games: int, search_strategy: callable) -> None:
    max_move_to_end = float('-inf')
    min_move_to_end = float('inf')
    sum_move_to_end = 0
    sum_time_taken = 0
    end_game_count = 0

    def run_simulation_single(i):
        state = GrassCutterEnvironment.create_init_state(size, random_seed=i)
        
        start_time = time.time()
        end_state = search_strategy(state)
        end_time = time.time()

        return end_state, end_time - start_time
    
    with ThreadPoolExecutor() as executor:
        end_states = list(executor.map(run_simulation_single, range(total_games)))
    
    for end_state, time_taken in end_states:
        if GrassCutterEnvironment.is_terminal(end_state):
            end_game_count += 1
            max_move_to_end = max(max_move_to_end, end_state.move_times)
            min_move_to_end = min(min_move_to_end, end_state.move_times)
            sum_move_to_end += end_state.move_times
            sum_time_taken += time_taken

    print('Random search result:')
    print(f'\t      Total games: {total_games:,}')
    print(f'\t Total end states: {end_game_count:,}({end_game_count/total_games:.2%}%)')
    print(f'\t Max moves to end: {max_move_to_end:,}')
    print(f'\t Min moves to end: {min_move_to_end:,}')
    print(f'\tMean moves to end: {sum_move_to_end / end_game_count:,.2f}')
    print(f'\t Total time taken: {sum_time_taken:,.4f} seconds')
    print(f'\t   Avg time taken: {sum_time_taken / end_game_count:,.4f} seconds')

Use random search
- select random move from available moves
- not has any intelligence

In [33]:
def random_search(state: GrassCutterState, verbose: bool = False) -> GrassCutterState:
    while not GrassCutterEnvironment.is_terminal(state):
        valid_moves = GrassCutterEnvironment.get_valid_moves(state)
        move = random.choice(valid_moves)
        state = GrassCutterEnvironment.apply_move(state, move)
        if verbose:
            print(state)
            print('----------------')
    return state

run_simulation(5, 10000, random_search)

Random search result:
	      Total games: 10,000
	 Total end states: 10,000(100.00%%)
	 Max moves to end: 1,785
	 Min moves to end: 14
	Mean moves to end: 357.26
	 Total time taken: 52.6257 seconds
	   Avg time taken: 0.0053 seconds


Use BFS 
- guarantee to find the shortest move to win
- but it is slow
  - has no condition -> need to search all possible moves
  - due to this problem, I will
    1. limit the depth of BFS to 10
    2. only simulate for 60 times

In [29]:
from collections import deque
from functools import partial

def bfs_search(state: GrassCutterState, max_move: int, verbose: bool = False) -> GrassCutterState:
    queue = deque([state])
    while queue:
        state = queue.popleft()
        if GrassCutterEnvironment.is_terminal(state):
            return state
        valid_moves = GrassCutterEnvironment.get_valid_moves(state)
        for move in valid_moves:
            new_state = GrassCutterEnvironment.apply_move(state, move)
            if new_state.move_times <= max_move:
                queue.append(new_state)
        if verbose:
            print(state)
            print('----------------')
    return state

bfs_search = partial(bfs_search, max_move=10)
run_simulation(5, 60, bfs_search)

Random search result:
	      Total games: 60
	 Total end states: 1(1.67%%)
	 Max moves to end: 9
	 Min moves to end: 9
	Mean moves to end: 9.00
	 Total time taken: 6.4652 seconds
	   Avg time taken: 6.4652 seconds


Use BFS but with some optimization
- guarantee to find the shortest move to win
- optimization
  1. will not simulate the same state twice

In [34]:
from collections import deque
from functools import partial

def bfs_search_optimized(state: GrassCutterState, max_move: int, verbose: bool = False) -> GrassCutterState:
    visited_states = set()
    queue = deque([state])
    while queue:
        state = queue.popleft()
        # Check if state is terminal
        if GrassCutterEnvironment.is_terminal(state):
            return state
        # Add new states to the queue
        valid_moves = GrassCutterEnvironment.get_valid_moves(state)
        for move in valid_moves:
            new_state = GrassCutterEnvironment.apply_move(state, move)
            if new_state.move_times <= max_move:
                # Check if new state is visited, if not add to queue and visited set
                if new_state not in visited_states:
                    visited_states.add(new_state)
                    queue.append(new_state)
        if verbose:
            print(state)
            print('----------------')
    return state

bfs_search_optimized = partial(bfs_search_optimized, max_move=float('inf'))
run_simulation(5, 10000, bfs_search_optimized)

Random search result:
	      Total games: 10,000
	 Total end states: 10,000(100.00%%)
	 Max moves to end: 21
	 Min moves to end: 5
	Mean moves to end: 14.46
	 Total time taken: 279.7888 seconds
	   Avg time taken: 0.0280 seconds


Use grass amount quality function
- optimization
  1. will not simulate the same state twice
  2. use quality function to select the best move to simulate

In [47]:
# Import priority queue
from queue import PriorityQueue

def inverse_grass_amount_quality_function(state: GrassCutterState) -> int:
    return - len(state.grass_position)

def quality_based_search(state: GrassCutterState, quality_function: callable, verbose: bool = False) -> GrassCutterState:
    visited_states = set()
    queue = PriorityQueue()
    queue.put((-quality_function(state), state))
    while not queue.empty():
        state = queue.get()[1]
        # Check if state is terminal
        if GrassCutterEnvironment.is_terminal(state):
            return state
        # Add new states to the queue
        valid_moves = GrassCutterEnvironment.get_valid_moves(state)
        for move in valid_moves:
            new_state = GrassCutterEnvironment.apply_move(state, move)
            if new_state not in visited_states:
                visited_states.add(new_state)
                queue.put((-quality_function(new_state), new_state))
        if verbose:
            print(state)
            print('----------------')
    return state

quality_based_search = partial(quality_based_search, quality_function=inverse_grass_amount_quality_function)
run_simulation(10, 10000, quality_based_search)

Random search result:
	      Total games: 10,000
	 Total end states: 10,000(100.00%%)
	 Max moves to end: 63
	 Min moves to end: 23
	Mean moves to end: 42.11
	 Total time taken: 178.6515 seconds
	   Avg time taken: 0.0179 seconds


Use grase amount and move amount quality function
- optimization
  1. will not simulate the same state twice
  2. use quality function to select the best move to simulate

In [ ]:
# Import priority queue
from queue import PriorityQueue

def optimized_quality_function(state: GrassCutterState) -> int:
    # Compute inverse grass amount quality function
    inverse_grass_amount = - len(state.grass_position)

    # Compute inverse move times quality function
    inverse_move_times = - state.move_times

    return inverse_grass_amount + inverse_move_times * 20
    

def quality_based_search(state: GrassCutterState, quality_function: callable, verbose: bool = False) -> GrassCutterState:
    visited_states = set()
    queue = PriorityQueue()
    queue.put((-quality_function(state), state))
    while not queue.empty():
        state = queue.get()[1]
        # Check if state is terminal
        if GrassCutterEnvironment.is_terminal(state):
            return state
        # Add new states to the queue
        valid_moves = GrassCutterEnvironment.get_valid_moves(state)
        for move in valid_moves:
            new_state = GrassCutterEnvironment.apply_move(state, move)
            if new_state not in visited_states:
                visited_states.add(new_state)
                queue.put((-quality_function(new_state), new_state))
        if verbose:
            print(state)
            print('----------------')
    return state

quality_based_search = partial(quality_based_search, quality_function=optimized_quality_function)
run_simulation(10, 100, quality_based_search)

Random search result:
	      Total games: 100
	 Total end states: 100(100.00%%)
	 Max moves to end: 47
	 Min moves to end: 28
	Mean moves to end: 39.42
	 Total time taken: 980.6797 seconds
	   Avg time taken: 9.8068 seconds
